In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import math
import pickle
import os
from datetime import datetime
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
random.seed(datetime.now().timestamp())

In [ ]:
num_of_nodes = [20,25,30,35,40,45,50,55,60]
test_case_num = 30

#defining network parameters
DEPLOY_AREA = (100,100) #sq.m
GATEWAY_ID = 0
TRANSMISSION_RANGE = 20 #m

In [ ]:
#Time on Air (ToA) is the amount of time that the transmitter antenna is energised and
#transmitting data.
#Note: ToA is not the time from Tx to Rx.

#Duty cycle is the proportion of time during which a component, device, or system is
# operated. The duty cycle can be expressed as a ratio or as a percentage.

# ToA = T_packet = T_preamble(sec) + T_payload(sec)
# T_preamble(sec) = (n_preamble + 4.25) T_s
# T_s = symbol duration /sweep time (sec) = (2^sf)/BW
# T_payload = T_s(8 + max(ceil((8PL-4SF+28+16CRC-20H)/4(SF-2DE)),(CR+4), 0))
# PL(payload) in bytes


#Defining LoRaWAN parameters

#DEFAULT VALUES (EU-868 values)
CRC = 1 
H = 0 
CR = 1
DE = 0 #LowDataRateOptimize (disabled)
BW = 250 #kHz
n_preamble = 8 
PL = 51 #bytes
ALPHA = 0.4
MIN_DATA_RATE = 100 #bps
duty_cycle = 0.1 #10% duty cycle
spread_factor = [7,8,9,10,11,12]

#Semtech SX1262IMLTRT LoRa radio transceiver
transmission_current = 4.3 #mA

#CR2450 Lithium battery
MAX_BATTERY_CAPACITY = 700 #mAh 
threshold_capacity = 0.5 #50%

#P1110B – 915 MHz RF Powerharvester
charge_current = 0.1*MAX_BATTERY_CAPACITY #mA

In [ ]:
#return time on air (millisec) for given spreaing factor
def time_on_air(SF):
  return ((n_preamble + 4.25) + (8 + max(math.ceil((8*PL - 4*SF + 28 + 16*CRC - 20*H)/(4*(SF-2*DE)))*(CR+4), 0)))*((2**SF)/BW)

#return sleep time in hours.
def sleep_time(SF):
  return (1-duty_cycle)*(time_on_air(SF)/(3600 * 1000))

#returns charging time in hours.
def charging_time(res_energy):
  return ((threshold_capacity*MAX_BATTERY_CAPACITY)-res_energy)/charge_current

def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return math.sqrt((x2-x1)**2+(y2-y1)**2)

#return transmission time in hours
tx_time = {k: duty_cycle*time_on_air(k)/(3600 * 1000) for k in spread_factor}

# return maximum edge datarate in bits/sec
max_edge_data_rate = {k: int(k*(BW/2**k)*(4/(4+CR))*1000) for k in spread_factor}

#maximum sleep time in hours
max_sleep_time = max([sleep_time(k) for k in spread_factor])

#maximum charging time in hours
max_charging_time = ((threshold_capacity*MAX_BATTERY_CAPACITY)-0.2*MAX_BATTERY_CAPACITY)/charge_current

#maximum transmission time in hours
max_tx_time = max([tx_time[k] for k in spread_factor])

In [ ]:
def get_all_paths(G, edge_nodes):
  all_paths = {}
  for source in edge_nodes:
      all_paths[source] = []
      paths = nx.all_simple_paths(G,source,GATEWAY_ID)
      for path in paths:
        prev = path[0]
        path_edge_list = []
        for node in path[1:]:
          path_edge_list.append((prev,node))
          prev = node
        all_paths[source].append(path_edge_list)

In [ ]:
def save_test_case(path, file_name, data, isGraph):
  if not os.path.exists(path):
    os.makedirs(path)
  if isGraph:
    #write adj list to a file
    nx.write_gml(data, path + "/" + file_name)
  else:
    with open(path + "/" + file_name,'wb') as file:
      pickle.dump(data, file)
      file.close()
  

In [ ]:

for N_DEVICES in num_of_nodes:
  for t in range(test_case_num):
  
    #create graph object
    G = nx.Graph()
    connected = False
    positions = {} #adding gateway position

    while len(G.nodes) != N_DEVICES or not connected:
      positions_set = set({})
      positions = {}
      for i in range(N_DEVICES):
        if i == GATEWAY_ID:
          continue
        (x,y) = (random.randint(-DEPLOY_AREA[0]//2, DEPLOY_AREA[0]//2),random.randint(-DEPLOY_AREA[0]//2, DEPLOY_AREA[0]//2))
        while (x,y) in positions_set:
          (x,y) = (random.randint(-DEPLOY_AREA[0]//2, DEPLOY_AREA[0]//2),random.randint(-DEPLOY_AREA[0]//2, DEPLOY_AREA[0]//2))
        positions_set.add((x,y))
        positions[i] = (x,y)

      positions[GATEWAY_ID] = (0,0)  #Gateway node has id 0
      G = nx.random_geometric_graph(N_DEVICES, TRANSMISSION_RANGE, pos = positions)
      connected = nx.is_connected(G)

    #Get the end devices
    import numpy as np
    from scipy.spatial import ConvexHull, convex_hull_plot_2d
    points = []
    for k in range(N_DEVICES):
      points.append(positions[k])
    edge_nodes = list(ConvexHull(points).vertices)
    convex_hull = np.array([positions[i] for i in edge_nodes])

    #define initial residual energy for each device
    init_res_energy = [random.randint(0.2*MAX_BATTERY_CAPACITY, MAX_BATTERY_CAPACITY) for i in range(N_DEVICES)]

    #LoRa Transmission Parameter Selection (paper): Transmission power range of lora devices is 2dBm to 20dBm
    #Assign the initial transmission power(dBm) for all the devices
    tx_pwr = [random.randint(2,20) for i in range(N_DEVICES)]

    #Assign the energy harvesting time (in hrs) for all the devices.
    eh_time = {(u,k) : max(sleep_time(k), charging_time(init_res_energy[u])) for u in range(N_DEVICES) for k in spread_factor}

    #get the updated residual energy of all the EH-devices
    res_eg = {(u,k): init_res_energy[u] + charge_current*eh_time[u,k] for u in range(N_DEVICES) for k in spread_factor}


    #creating a list of bidirectional edges
    edge_list = []
    for u,v in G.edges:
      edge_list.append((u,v))
      edge_list.append((v,u))

    #Assign data rate for each edge in bits/sec
    data_rate = {(i,u,v,k): random.randint(MIN_DATA_RATE, max_edge_data_rate[k]) for i in edge_nodes for u,v in edge_list for k in spread_factor}

    #calculate max edge delay: max_tx_delay + eh_time + 2*(dist/speed) hours
    speed = 3 * (10**8) #m/s
    max_edge_delays = {(u,v): max_tx_time + max(max_sleep_time, max_charging_time) + 2*(dist(positions[u],positions[v])/speed)/3600
                   for u,v in edge_list}

    #assign delay (propagation) in hrs for each edge in hours
    edge_delays = {(i,u,v): random.uniform((dist(positions[u],positions[v])/speed)/3600, 2*(dist(positions[u],positions[v])/speed)/3600) for i in edge_nodes for u,v in edge_list}

    #get all possible path to the gateway for each flow
    # all_paths = get_all_paths(G, edge_nodes)
    
    #calculate max delays for each flow in hours
    max_flow_delays = {}
    for i in edge_nodes:
      # path_lengths = [len(path) for path in all_paths[i]]
      # avg_path_len = sum(path_lengths)/len(path_lengths)
      # max_flow_delays[i] = (max(max_edge_delays.values()))*avg_path_len
      max_flow_delays[i] = (max(max_edge_delays.values()))*int(N_DEVICES/3)

    cst = {(i,u,v,k): ALPHA*((edge_delays[i,u,v] + tx_time[k] + eh_time[u,k])/max_edge_delays[u,v]) + (1-ALPHA)*((init_res_energy[u]-transmission_current*tx_time[k]+charge_current*eh_time[u,k])/MAX_BATTERY_CAPACITY)
      for i in edge_nodes for u,v in edge_list for k in spread_factor}

    print(f"num_nodes = {N_DEVICES} graph-{t+1}")

    # #visualizing the graph
    # color_map = ["blue" for i in range(N_DEVICES)]
    # color_map[GATEWAY_ID] = "red"
    # plt.figure(figsize=(10,10))
    # nx.draw_networkx(G, positions,node_size = 300, node_color=color_map, font_color = "white", with_labels = True)
    # plt.show()

    #save graph data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "graph.gml", G, True)
    

    #writing tx_time to a file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "tx_time.txt", tx_time, False)

    #writing max_edge_data_rate to a file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "max_edge_data_rate.txt", max_edge_data_rate, False)

    #writing cost data to a file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "cst.txt", cst, False)

    #writing max_flow_delay data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "max_flow_delays.txt", max_flow_delays, False)

    #writing edge_delays data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "edge_delays.txt", edge_delays, False)

    #writing max_edge delay data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "max_edge_delays.txt", max_edge_delays, False)

    #writing edge datarate data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "data_rate.txt", data_rate, False)

    #writing eh time to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "eh_time.txt", eh_time, False)

    #writing initial residual energy data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "init_res_energy.txt", init_res_energy, False) 

    #writing edge nodes data to file
    save_test_case(f'test_cases/num_nodes_{N_DEVICES}/test_{t+1}', "edge_nodes.txt", edge_nodes, False) 

In [ ]:
!zip -r /content/gdrive/MyDrive/Networks/test_cases.zip /content/test_cases